In [5]:
!pip install faiss-cpu torch pypdf
!pip install -U langchain-community
!pip install sentence_transformers
!pip install bitsandbytes

In [6]:
from google.colab import drive
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import bitsandbytes as bnb
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Step 1: Load the IPC Document
pdf_path = "/content/drive/MyDrive/ipc_act.pdf"  # Ensure you have the official IPC PDF
document_loader = PyPDFLoader(pdf_path)
documents = document_loader.load()

In [8]:
# Step 2: Process and Split the Text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

In [2]:
!pip install langchain_huggingface

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

In [9]:
# Step 3: Create a FAISS Vector Store (using CPU version, optimized for memory)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(split_docs, embeddings)
retriever = vector_store.as_retriever()

<ipython-input-9-c68b1abc7b24>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
# Step 4: Load a Smaller Free LLM from Hugging Face (Optimized for RAM)
model_name = "tiiuae/falcon-7b-instruct"  # Smaller and optimized model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True  # Enables 4-bit quantization for speedup
)
nlp_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)

llm = HuggingFacePipeline(pipeline=nlp_pipeline)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
<ipython-input-10-9fb6bd88f649>:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=nlp_pipeline)


In [11]:
from langchain.memory import ConversationBufferMemory


In [39]:
from langchain.chains import RetrievalQA

# Initialize the chain using the correct input key
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [41]:
def chat():
    print("IPC Chatbot: Ask me about the Indian Penal Code!")
    while True:
        query = input("You: ")
        if query.lower() in ['exit', 'quit', 'bye']:
            print("IPC Chatbot: Goodbye!")
            break

        try:
            # Use 'query' since RetrievalQA expects it
            response = qa_chain.invoke({"query": query})

            # RetrievalQA returns a direct response, no need for 'get'
            answer = response.get('result', 'No answer available')
            print("\nIPC Chatbot Response:\n" )
            print(answer)
            print("-"*30)


        except Exception as e:
            print("Error:", e)

chat()

IPC Chatbot: Ask me about the Indian Penal Code!
You: what is the punishment for murder?


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.

IPC Chatbot Response:

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

or knew himself to he likely to cause. 
302. 
Punishment for murder. 
302. Punishment for murder.--Whoever commits murder shall be 
punished with death, or 1*[imprisonment for life], and shall also 
be liable to fine. 
303. 
Punishment for murder by life-convict. 
303. Punishment for murder by life-convict.--Whoever, being under 
sentence of 1*[imprisonment for life], commits murder, shall be 
punished with death. 
304. 
Punishment for culpable homicide not amounting to murder. 
304. Punishment for culpable homicide not amounting to murder.- -
Whoever commits culpable homicide not amounting to murder shall be 
punished with 1*[imprisonment for life], or imprisonment of either 
description for a term which may extend to ten years, and shall also 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.

IPC Chatbot Response:

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

be so disposed of as to be put in danger of being murdered, shall be 
punished with 1*[imprisonment for life] or rigorous imprisonment 
for a term which may extend to ten years, and shall also be liable to 
fine.

or knowledge, and under such circumstances that, if he by that act 
caused death, he would be guilty of murder, shall be punished with 
imprisonment of either description for a term which may extend to ten 
years, and shall also be liable to fine; and, if hurt is caused to any 
person by such act, the offender shall be liable either to 
1*[imprisonment for life], or to such punishment as is hereinbefore 
mentioned. 
Attempts by life-convicts. 
Attempts by life-convicts.-2*[When any person offending under 
this section is under sentence